In [4]:
import torch

depth = 24
ckpt_path = "/mnt/sda1/tkerssies/beit3_large_patch16_224.pth"

ckpt = torch.load(ckpt_path)["model"]

ckpt = {k.replace("beit3.", ""): v for k, v in ckpt.items()}
ckpt = {k: v for k, v in ckpt.items() if not k.startswith("text_embed")}
ckpt = {k.replace("vision_embed.", "patch_embed."): v for k, v in ckpt.items()}
ckpt.pop("patch_embed.mask_token")
ckpt.pop("mlm_head.weight")
ckpt.pop("mlm_head.bias")
ckpt.pop("mim_head.weight")
ckpt.pop("mim_head.bias")
ckpt = {k.replace("patch_embed.cls_token", "cls_token"): v for k, v in ckpt.items()}
ckpt = {k.replace("encoder.", ""): v for k, v in ckpt.items()}
ckpt = {k.replace("layers", "blocks"): v for k, v in ckpt.items()}
ckpt = {k: v for k, v in ckpt.items() if ".B." not in k}
ckpt = {k.replace(".A.", "."): v for k, v in ckpt.items()}
ckpt["pos_embed"] = ckpt.pop("embed_positions.weight")
ckpt = {k.replace("self_attn_layer_norm", "norm1"): v for k, v in ckpt.items()}
ckpt = {k.replace("self_attn", "attn"): v for k, v in ckpt.items()}
ckpt = {k.replace("inner_attn_ln", "proj.0"): v for k, v in ckpt.items()}
ckpt = {k.replace("out_proj", "proj.1"): v for k, v in ckpt.items()}
ckpt = {k.replace("ffn", "mlp"): v for k, v in ckpt.items()}
ckpt = {k.replace("final_layer_norm", "norm2"): v for k, v in ckpt.items()}
ckpt = {k.replace("mlp_layernorm", "norm"): v for k, v in ckpt.items()}
ckpt["norm.weight"] = ckpt.pop("layer_norm.weight")
ckpt["norm.bias"] = ckpt.pop("layer_norm.bias")
ckpt["pos_embed"] = ckpt["pos_embed"][None, 2:, :]

for block in range(depth):
    q_key = f"blocks.{block}.attn.q_proj"
    k_key = f"blocks.{block}.attn.k_proj"
    v_key = f"blocks.{block}.attn.v_proj"

    q_w = ckpt.pop(f"{q_key}.weight")
    k_w = ckpt.pop(f"{k_key}.weight")
    v_w = ckpt.pop(f"{v_key}.weight")

    q_b = ckpt.pop(f"{q_key}.bias")
    k_b = ckpt.pop(f"{k_key}.bias")
    v_b = ckpt.pop(f"{v_key}.bias")

    qkv_w = torch.cat([q_w, k_w, v_w], dim=0)
    qkv_b = torch.cat([q_b, k_b, v_b])

    ckpt[f"blocks.{block}.attn.qkv.weight"] = qkv_w
    ckpt[f"blocks.{block}.attn.qkv.bias"] = qkv_b

print("\n".join(ckpt.keys()))

torch.save(ckpt, ckpt_path + ".timm")

module.positional_embedding
module.text_projection
module.logit_scale
module.visual.class_embedding
module.visual.positional_embedding
module.visual.proj
module.visual.conv1.weight
module.visual.ln_pre.weight
module.visual.ln_pre.bias
module.visual.transformer.resblocks.0.ln_1.weight
module.visual.transformer.resblocks.0.ln_1.bias
module.visual.transformer.resblocks.0.attn.in_proj_weight
module.visual.transformer.resblocks.0.attn.in_proj_bias
module.visual.transformer.resblocks.0.attn.out_proj.weight
module.visual.transformer.resblocks.0.attn.out_proj.bias
module.visual.transformer.resblocks.0.ln_2.weight
module.visual.transformer.resblocks.0.ln_2.bias
module.visual.transformer.resblocks.0.mlp.c_fc.weight
module.visual.transformer.resblocks.0.mlp.c_fc.bias
module.visual.transformer.resblocks.0.mlp.c_proj.weight
module.visual.transformer.resblocks.0.mlp.c_proj.bias
module.visual.transformer.resblocks.1.ln_1.weight
module.visual.transformer.resblocks.1.ln_1.bias
module.visual.transformer